In [3]:
#import necessary libraries
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import xarray as xr
import scipy

import regionmask

# import regionmask

from scipy import stats
from scipy.stats import pearsonr, linregress

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd

import time
from datetime import datetime, date

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.ticker import MaxNLocator, FuncFormatter
import matplotlib.dates as mdates

import shapefile
from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings
import os

import sys
import os.path
import subprocess
import math
import csv

from netCDF4 import Dataset

#import s3fs
import copy
import json, copy
from pandas import DataFrame
from osgeo import gdal, ogr, osr

"D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\Mascon_gsfc_pdsmc_tws1_meter.nc"

"D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\Mascon_jpl_pdsmc_tws1_meter.nc"

"D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\Mascon_csr_pdsmc_tws1_meter.nc"

In [4]:
# Check if the file was properly saved at the given location (navigate to the folder) and open it from there again
gsfc_pdsmc_tws = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\Mascon_gsfc_pdsmc_tws1_meter.nc')
jpl_pdsmc_tws = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\Mascon_jpl_pdsmc_tws1_meter.nc')
csr_pdsmc_tws = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\Mascon_csr_pdsmc_tws1_meter.nc')

In [5]:
gsfc_pdsmc_tws

<xarray.Dataset> Size: 348MB
Dimensions:        (lon: 720, lat: 360, time: 168)
Coordinates:
  * lon            (lon) float64 6kB 0.0 0.5 1.0 1.5 ... 358.0 358.5 359.0 359.5
  * lat            (lat) float64 3kB -90.0 -89.5 -89.0 -88.5 ... 88.5 89.0 89.5
  * time           (time) datetime64[ns] 1kB 2002-04-30 ... 2016-03-31
Data variables:
    lwe_thickness  (time, lat, lon) float64 348MB ...

In [6]:
gsfc = gsfc_pdsmc_tws['lwe_thickness'].values
jpl = jpl_pdsmc_tws['lwe_thickness'].values
csr = csr_pdsmc_tws['lwe_thickness'].values

In [ ]:
csr

In [36]:
def vce_weighted_average_monthly(solutions_monthly, max_iter=50):
    n_solutions = solutions_monthly.shape[0]
    weights = np.ones(n_solutions) / n_solutions # Start with equal weights, i,e. 1/3, 1/3, 1/3 for GSFC, JPL & CSR
    for _ in range(max_iter):
        # Weighted average using current weights
        combined = np.average(solutions_monthly, axis=0, weights=weights)

        # Calculate residuals for each solution
        residuals = solutions_monthly - combined

        # Calculate RMS of residuals for each solution
        rms = np.sqrt(np.mean(residuals**2, axis=(1,2)))
        rms[rms == 0] = 1e-12 # to prevent division by zero

        
        # --- weight update as in Eq. (2) as in the paper: https://doi.org/10.1007/1345_2022_163 -------
        sum_weights = np.sum(weights)
        normalized_prev_weights = weights / sum_weights
        one_minus_normalized = 1.0 - normalized_prev_weights
        new_weights = one_minus_normalized * (1.0 / (rms ** 2))
        new_weights /= new_weights.sum()  # Normalize the weight so that the weight of 3 data providers sum to 1

       # Check convergence of new weight to fifth decimal place 
        if np.all(np.round(weights, 5) == np.round(new_weights, 5)):
            break
            
        weights = new_weights
        
    return combined, weights

# Stack all solutions: shape (3, 168, 360, 720)
all_solutions = np.stack([gsfc, jpl, csr], axis=0)  # shape (3, 168, 360, 720)
n_months = all_solutions.shape[1]
weights_all = np.zeros((n_months, 3), dtype=np.float32)

for t in range(n_months):
    monthly_solutions = all_solutions[:, t, :, :]
    _, weights = vce_weighted_average_monthly(monthly_solutions)
    weights_all[t] = weights

# Compute overall weights for the entire period: April 2002 - march 2016 : 14 yr period
overall_weights = np.mean(weights_all, axis=0)
print("Overall weights for GSFC, JPL, CSR (to 5 decimals):")
print(np.round(overall_weights, 5))

Overall weights for GSFC, JPL, CSR (to 5 decimals):
[0.45038 0.22347 0.32615]


In [ ]:
weights_gsfc = 0.45038
weights_jpl = 0.22347
weights_csr = 0.32615

In [ ]:
# Check if the file was properly saved at the given location (navigate to the folder) and open it from there again
gsfc_pdsmc_tws = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\Mascon_gsfc_pdsmc_tws1_meter.nc')
jpl_pdsmc_tws = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\Mascon_jpl_pdsmc_tws1_meter.nc')
csr_pdsmc_tws = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\Mascon_csr_pdsmc_tws1_meter.nc')

In [ ]:
csr_pdsmc_tws

In [9]:
weights_gsfc = 0.45038
weights_jpl = 0.22347
weights_csr = 0.32615

numerator = (
    jpl_pdsmc_tws['lwe_thickness'] * weights_jpl +
    gsfc_pdsmc_tws['lwe_thickness'] * weights_gsfc +
    csr_pdsmc_tws['lwe_thickness'] * weights_csr
)
denominator = weights_jpl + weights_gsfc + weights_csr
EWA_mascon = numerator / denominator

In [10]:
EWA_mascon

<xarray.DataArray 'lwe_thickness' (time: 168, lat: 360, lon: 720)> Size: 348MB
array([[[-0.06491064, -0.06485434, -0.06479799, ..., -0.06507932,
         -0.06502313, -0.06496691],
        [-0.06541341, -0.06529208, -0.06517231, ..., -0.06578618,
         -0.06566051, -0.06553623],
        [-0.0540679 , -0.05390322, -0.05374313, ..., -0.05458875,
         -0.05441074, -0.05423709],
        ...,
        [-0.01536741, -0.01536741, -0.01536741, ..., -0.01751804,
         -0.01751804, -0.01751804],
        [-0.01536741, -0.01536741, -0.01536741, ..., -0.01751804,
         -0.01751804, -0.01751804],
        [-0.01745213, -0.01745213, -0.01745213, ..., -0.01745213,
         -0.01745213, -0.01745213]],

       [[-0.06165628, -0.06160057, -0.06154485, ..., -0.06182336,
         -0.06176768, -0.06171198],
        [-0.06124761, -0.06112703, -0.06100905, ..., -0.06162439,
         -0.06149634, -0.06137073],
        [-0.04917329, -0.04899837, -0.04882966, ..., -0.04973453,
         -0.04954146, -0.04935435],
...
        [-0.02145995, -0.02145995, -0.02145995, ..., -0.01744064,
         -0.01744064, -0.01744064],
        [-0.02145995, -0.02145995, -0.02145995, ..., -0.01744064,
         -0.01744064, -0.01744064],
        [-0.01329658, -0.01329658, -0.01329658, ..., -0.01329658,
         -0.01329658, -0.01329658]],

       [[ 0.1117972 ,  0.111706  ,  0.11161412, ...,  0.11206736,
          0.11197787,  0.11188783],
        [ 0.11716571,  0.11698784,  0.11681141, ...,  0.1177075 ,
          0.11752559,  0.11734498],
        [ 0.09855698,  0.09834508,  0.09814173, ...,  0.09924395,
          0.09900646,  0.09877745],
        ...,
        [-0.02604074, -0.02604074, -0.02604074, ..., -0.02342865,
         -0.02342865, -0.02342865],
        [-0.02604074, -0.02604074, -0.02604074, ..., -0.02342865,
         -0.02342865, -0.02342865],
        [-0.01584726, -0.01584726, -0.01584726, ..., -0.01584726,
         -0.01584726, -0.01584726]]], shape=(168, 360, 720))
Coordinates:
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5
  * lat      (lat) float64 3kB -90.0 -89.5 -89.0 -88.5 ... 88.0 88.5 89.0 89.5
  * time     (time) datetime64[ns] 1kB 2002-04-30 2002-05-31 ... 2016-03-31

In [11]:
EWA_mascon.attrs['grid_mapping'] = 'WGS84'
EWA_mascon.attrs['standard_name'] = 'Liquid_Water_Equivalent_Thickness'
EWA_mascon.attrs['long_name'] = 'Liquid_Water_Equivalent_Thickness'
EWA_mascon.attrs['units'] = 'meters'

# # Now export to NetCDF
# EWA_mascon.to_netcdf('EWA_mascon.nc')

In [12]:
EWA_mascon

<xarray.DataArray 'lwe_thickness' (time: 168, lat: 360, lon: 720)> Size: 348MB
array([[[-0.06491064, -0.06485434, -0.06479799, ..., -0.06507932,
         -0.06502313, -0.06496691],
        [-0.06541341, -0.06529208, -0.06517231, ..., -0.06578618,
         -0.06566051, -0.06553623],
        [-0.0540679 , -0.05390322, -0.05374313, ..., -0.05458875,
         -0.05441074, -0.05423709],
        ...,
        [-0.01536741, -0.01536741, -0.01536741, ..., -0.01751804,
         -0.01751804, -0.01751804],
        [-0.01536741, -0.01536741, -0.01536741, ..., -0.01751804,
         -0.01751804, -0.01751804],
        [-0.01745213, -0.01745213, -0.01745213, ..., -0.01745213,
         -0.01745213, -0.01745213]],

       [[-0.06165628, -0.06160057, -0.06154485, ..., -0.06182336,
         -0.06176768, -0.06171198],
        [-0.06124761, -0.06112703, -0.06100905, ..., -0.06162439,
         -0.06149634, -0.06137073],
        [-0.04917329, -0.04899837, -0.04882966, ..., -0.04973453,
         -0.04954146, -0.04935435],
...
        [-0.02145995, -0.02145995, -0.02145995, ..., -0.01744064,
         -0.01744064, -0.01744064],
        [-0.02145995, -0.02145995, -0.02145995, ..., -0.01744064,
         -0.01744064, -0.01744064],
        [-0.01329658, -0.01329658, -0.01329658, ..., -0.01329658,
         -0.01329658, -0.01329658]],

       [[ 0.1117972 ,  0.111706  ,  0.11161412, ...,  0.11206736,
          0.11197787,  0.11188783],
        [ 0.11716571,  0.11698784,  0.11681141, ...,  0.1177075 ,
          0.11752559,  0.11734498],
        [ 0.09855698,  0.09834508,  0.09814173, ...,  0.09924395,
          0.09900646,  0.09877745],
        ...,
        [-0.02604074, -0.02604074, -0.02604074, ..., -0.02342865,
         -0.02342865, -0.02342865],
        [-0.02604074, -0.02604074, -0.02604074, ..., -0.02342865,
         -0.02342865, -0.02342865],
        [-0.01584726, -0.01584726, -0.01584726, ..., -0.01584726,
         -0.01584726, -0.01584726]]], shape=(168, 360, 720))
Coordinates:
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5
  * lat      (lat) float64 3kB -90.0 -89.5 -89.0 -88.5 ... 88.0 88.5 89.0 89.5
  * time     (time) datetime64[ns] 1kB 2002-04-30 2002-05-31 ... 2016-03-31
Attributes:
    grid_mapping:   WGS84
    standard_name:  Liquid_Water_Equivalent_Thickness
    long_name:      Liquid_Water_Equivalent_Thickness
    units:          meters

In [78]:
EWA_mascon.to_netcdf(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\VCE_EWA_mascon_pdsmc_tws1_meter_attribute2.nc')

In [7]:
weighted_mean_mascon = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\VCE_EWA_mascon_pdsmc_tws1_meter_attribute2.nc')
weighted_mean_mascon

<xarray.Dataset> Size: 348MB
Dimensions:        (lon: 720, lat: 360, time: 168)
Coordinates:
  * lon            (lon) float64 6kB 0.0 0.5 1.0 1.5 ... 358.0 358.5 359.0 359.5
  * lat            (lat) float64 3kB -90.0 -89.5 -89.0 -88.5 ... 88.5 89.0 89.5
  * time           (time) datetime64[ns] 1kB 2002-04-30 ... 2016-03-31
Data variables:
    lwe_thickness  (time, lat, lon) float64 348MB ...

In [8]:
weighted_mean_mascon.lwe_thickness

<xarray.DataArray 'lwe_thickness' (time: 168, lat: 360, lon: 720)> Size: 348MB
[43545600 values with dtype=float64]
Coordinates:
  * lon      (lon) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5
  * lat      (lat) float64 3kB -90.0 -89.5 -89.0 -88.5 ... 88.0 88.5 89.0 89.5
  * time     (time) datetime64[ns] 1kB 2002-04-30 2002-05-31 ... 2016-03-31
Attributes:
    grid_mapping:   WGS84
    standard_name:  Liquid_Water_Equivalent_Thickness
    long_name:      Liquid_Water_Equivalent_Thickness
    units:          meters

In [9]:
# 1. Load and prepare the shapefile
gdf = gpd.read_file(r'D:\MSRSGI\Masters_thesis\Data\HMA_Region_Shapefiles\Regions_shapefiles\New folder\Updated_TotalHMA_boundary1_22.shp')
gdf_4326 = gdf.to_crs('EPSG:4326')  # Reproject to match dataset CRS

dropped_gdf_4326 = gdf_4326.drop(['Primary__1'], axis=1)
dropped_gdf_4326

,Primary_ID,Area,Shape_Area,Region_No,geometry
0,Eastern Hindu Kush,111147.061871,1.111471e+11,8,"POLYGON ((73.45158 36.98397, 73.44585 36.96572..."
1,Western Himalaya,211514.200081,2.115142e+11,10,"POLYGON ((75.78739 35.3156, 75.7874 35.3156, 7..."
2,Eastern Himalaya,166943.561268,1.669436e+11,12,"POLYGON ((95.22303 29.82187, 95.21352 29.80717..."
3,Central Himalaya,270268.094462,2.702681e+11,11,"POLYGON ((80.93559 30.70763, 80.98095 30.6962,..."
4,Karakoram,92830.485352,9.283049e+10,9,"POLYGON ((75.87358 37.3118, 75.89056 37.31113,..."
5,Western Pamir,124880.500694,1.248805e+11,6,"POLYGON ((73.91513 39.69697, 73.92109 39.68454..."
6,Pamir Alay,83444.425531,8.344443e+10,5,"POLYGON ((73.66215 40.58146, 73.727 40.56312, ..."
7,Northern/Western Tien Shan,205724.916795,2.057249e+11,4,"POLYGON ((83.06636 43.4915, 82.95135 43.44983,..."
8,Dzhungarsky Alatau,39543.166059,3.954317e+10,1,"POLYGON ((81.62713 44.61643, 81.28999 44.60156..."
9,Western Kunlun Shan,132813.049169,1.328130e+11,13,"POLYGON ((77.76951 37.37377, 78.24539 37.24774..."


In [10]:
gdf1 = gpd.read_file(r"D:\MSRSGI\Masters_thesis\Data\HMA_Region_Shapefiles\Regions_shapefiles\New folder\Updated_SingleHMAboundarythatincludes1to22.shp")
gdf1_4326 = gdf1.to_crs('EPSG:4326')  # Reproject to match dataset CRS

dropped_gdf1_4326 = gdf1_4326.drop(['Primary__1'], axis=1)  #already done , so comes weror wehn done
dropped_gdf1_4326

,Primary_ID,Area,Shape_Area,Region_No,geometry
0,High Mountain Asia Region,3.993978e+06,3.993978e+12,23,"POLYGON ((92.26394 39.3763, 92.24847 39.32136,..."


In [11]:
gdf_concat = pd.concat([dropped_gdf_4326, dropped_gdf1_4326 ], axis=0, ignore_index=True)
gdf_concat

,Primary_ID,Area,Shape_Area,Region_No,geometry
0,Eastern Hindu Kush,1.111471e+05,1.111471e+11,8,"POLYGON ((73.45158 36.98397, 73.44585 36.96572..."
1,Western Himalaya,2.115142e+05,2.115142e+11,10,"POLYGON ((75.78739 35.3156, 75.7874 35.3156, 7..."
2,Eastern Himalaya,1.669436e+05,1.669436e+11,12,"POLYGON ((95.22303 29.82187, 95.21352 29.80717..."
3,Central Himalaya,2.702681e+05,2.702681e+11,11,"POLYGON ((80.93559 30.70763, 80.98095 30.6962,..."
4,Karakoram,9.283049e+04,9.283049e+10,9,"POLYGON ((75.87358 37.3118, 75.89056 37.31113,..."
5,Western Pamir,1.248805e+05,1.248805e+11,6,"POLYGON ((73.91513 39.69697, 73.92109 39.68454..."
6,Pamir Alay,8.344443e+04,8.344443e+10,5,"POLYGON ((73.66215 40.58146, 73.727 40.56312, ..."
7,Northern/Western Tien Shan,2.057249e+05,2.057249e+11,4,"POLYGON ((83.06636 43.4915, 82.95135 43.44983,..."
8,Dzhungarsky Alatau,3.954317e+04,3.954317e+10,1,"POLYGON ((81.62713 44.61643, 81.28999 44.60156..."
9,Western Kunlun Shan,1.328130e+05,1.328130e+11,13,"POLYGON ((77.76951 37.37377, 78.24539 37.24774..."


In [89]:
# weighted_mean_mascon = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\VCE_EWA_mascon_pdsmc_tws1_meter_attribute2.nc')
# weighted_mean_mascon

In [14]:
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray
from shapely.geometry import mapping

# # 1. Load and prepare the shapefile
# gdf = gpd.read_file(r'D:/MSRSGI/Masters_thesis/Data/HMA_Region_Shapefiles/Regions_shapefiles/HMA_region_annoted_final.shp')
# gdf_4326 = gdf.to_crs('EPSG:4326')  # Reproject to match dataset CRS


# 2. Load the lwe_thickness dataset 
ds = xr.open_dataset(r'D:\MSRSGI\Lab Rotation\Physical Geodesy\Python_Workspace\Organaized\Mascon\Combined\Result\Processed Data\VCE_EWA_mascon_pdsmc_tws1_meter_attribute2.nc')
da = ds['lwe_thickness']  # Get the DataArray

# 3. Define the provided functions
def area_weighting(grid_resolution):
    """
    Calculate the area of each grid, globally, corresponding to the latitudes and longitudes.
    """
    longitude_grid = np.linspace(0, 359+(1-grid_resolution), int(360/grid_resolution), dtype='float')
    latitude_grid = np.linspace(0, 179+(1-grid_resolution), int(180/grid_resolution), dtype='float')
    longitude_grid1 = np.linspace(1*grid_resolution, 360, int(360/grid_resolution), dtype='float')
    latitude_grid1 = np.linspace(1*grid_resolution, 180, int(180/grid_resolution), dtype='float')
    
    lambd,theta = np.meshgrid(longitude_grid, latitude_grid)  
    lambd1,theta1 = np.meshgrid(longitude_grid1, latitude_grid1)
    
    delta_latitude = np.sin(np.deg2rad(90-theta)) - np.sin(np.deg2rad(90-theta1))
    delta_longitude = (lambd1 - lambd) * np.pi/180

    # Area of each grid in m²
    area = (6378.137**2) * pow(10,6) * (np.multiply(delta_latitude, delta_longitude))
    return area

def Basinaverage(temp, gs, shp_basin, basin_area):
    """
    Calculate the basin average of the total water storage (TWS).
    """
    # Apply area weighting to the data
    temp_weighted = temp.copy()
    temp_weighted = temp * area_weighting(gs)  # Convert to volume
    
    # Set spatial dimensions and CRS
    basin_tws = temp_weighted.rio.write_crs("EPSG:4326", inplace=True)
    basin_tws = basin_tws.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
            
    # Clip data to basin shape
    basin_tws = basin_tws.rio.clip(shp_basin.geometry.apply(mapping), shp_basin.crs, drop=True)
    
    # Calculate basin average TWS
    basin_avg_tws = basin_tws.sum(dim=('lon','lat'), skipna=True) / basin_area
    
    # Clean up spatial references
    if 'spatial_ref' in basin_avg_tws.coords:
        basin_avg_tws = basin_avg_tws.drop_vars('spatial_ref')
    
    return basin_avg_tws

# 4. Process each region
results = []
grid_resolution = 0.5  # Grid resolution in degrees

#for idx, region in gdf_4326.iterrows():
for idx, region in gdf_concat.iterrows():
    region_name = region.Primary_ID
    region_area = region.Shape_Area  # Exact region area from shapefile
    number_region = region.Region_No #latest
    
    #  GeoDataFrame
    region_gdf = gpd.GeoDataFrame({
        'geometry': [region.geometry],
        'Primary_ID': [region_name],
        'Region_No': [number_region],
        'Shape_Area': [region_area]
    }, crs=gdf_concat.crs)   #gdf_4326 is replaced by gdf_concat
    
    # Calculate the basin average using the reference function
    basin_avg = Basinaverage(da, grid_resolution, region_gdf, region_area)
    
    # # Add region metadata
    # basin_avg = basin_avg.assign_attrs({
    #     'region_name': region_name,
    #     #'region_area_m2': float(region_area),
    #     'region_area_m2': region_area,
    #     'region_id': number_region
    # })

    meta = xr.Dataset({
    'region_id': result_ds.region_id,
    'region_area_m2': result_ds.region_area_m2}, coords={'region': result_ds.region})

    # print(meta.to_dataframe())
    
    basin_avg = basin_avg.expand_dims(region=[region_name])
    basin_avg = basin_avg.assign_coords({
    'region_area_m2': ('region', [region_area]),
    'region_id': ('region', [number_region])})
    results.append(basin_avg)

# 5. Combine all regions into a single dataset
result_ds = xr.concat(results, dim='region')

In [15]:
result_ds

<xarray.DataArray 'lwe_thickness' (region: 23, time: 168)> Size: 31kB
array([[ 0.08477645,  0.1055823 ,  0.05362286, ..., -0.09298977,
        -0.10053499,  0.04244633],
       [ 0.09492369,  0.08893011,  0.06801417, ..., -0.0398426 ,
        -0.03777494, -0.02573085],
       [ 0.014022  ,  0.03583394,  0.09582066, ..., -0.23747176,
        -0.26758587, -0.34150973],
       ...,
       [ 0.05585885,  0.08123649,  0.07697072, ..., -0.1635807 ,
        -0.13094854, -0.13640374],
       [ 0.00550206,  0.01609112,  0.0053254 , ..., -0.01568992,
        -0.02752489, -0.03344602],
       [ 0.01167397,  0.0134757 ,  0.01934877, ..., -0.0800768 ,
        -0.07820227, -0.08021396]], shape=(23, 168))
Coordinates:
  * region          (region) object 184B 'Eastern Hindu Kush' ... 'High Mount...
  * time            (time) datetime64[ns] 1kB 2002-04-30 ... 2016-03-31
    region_area_m2  (region) float64 184B 1.111e+11 2.115e+11 ... 3.994e+12
    region_id       (region) int64 184B 8 10 12 11 9 6 5 4 ... 22 14 15 2 3 7 23

In [16]:
result_ds_mm = result_ds * 1000
result_ds_mm

<xarray.DataArray 'lwe_thickness' (region: 23, time: 168)> Size: 31kB
array([[  84.77645061,  105.58229918,   53.62285924, ...,  -92.98976717,
        -100.53499218,   42.44632627],
       [  94.92369147,   88.93011344,   68.0141734 , ...,  -39.84260384,
         -37.77494184,  -25.73085421],
       [  14.02199857,   35.83394149,   95.82066059, ..., -237.47175627,
        -267.58587267, -341.50973328],
       ...,
       [  55.85885325,   81.23649365,   76.97071571, ..., -163.58069798,
        -130.94854008, -136.40374388],
       [   5.50205986,   16.09112216,    5.3253959 , ...,  -15.68991606,
         -27.52488534,  -33.44602371],
       [  11.67396517,   13.47570328,   19.34877473, ...,  -80.07679956,
         -78.20226625,  -80.21395638]], shape=(23, 168))
Coordinates:
  * region          (region) object 184B 'Eastern Hindu Kush' ... 'High Mount...
  * time            (time) datetime64[ns] 1kB 2002-04-30 ... 2016-03-31
    region_area_m2  (region) float64 184B 1.111e+11 2.115e+11 ... 3.994e+12
    region_id       (region) int64 184B 8 10 12 11 9 6 5 4 ... 22 14 15 2 3 7 23

In [17]:
result_ds_mm.attrs['units'] = 'mm'

In [18]:
result_ds_mm

<xarray.DataArray 'lwe_thickness' (region: 23, time: 168)> Size: 31kB
array([[  84.77645061,  105.58229918,   53.62285924, ...,  -92.98976717,
        -100.53499218,   42.44632627],
       [  94.92369147,   88.93011344,   68.0141734 , ...,  -39.84260384,
         -37.77494184,  -25.73085421],
       [  14.02199857,   35.83394149,   95.82066059, ..., -237.47175627,
        -267.58587267, -341.50973328],
       ...,
       [  55.85885325,   81.23649365,   76.97071571, ..., -163.58069798,
        -130.94854008, -136.40374388],
       [   5.50205986,   16.09112216,    5.3253959 , ...,  -15.68991606,
         -27.52488534,  -33.44602371],
       [  11.67396517,   13.47570328,   19.34877473, ...,  -80.07679956,
         -78.20226625,  -80.21395638]], shape=(23, 168))
Coordinates:
  * region          (region) object 184B 'Eastern Hindu Kush' ... 'High Mount...
  * time            (time) datetime64[ns] 1kB 2002-04-30 ... 2016-03-31
    region_area_m2  (region) float64 184B 1.111e+11 2.115e+11 ... 3.994e+12
    region_id       (region) int64 184B 8 10 12 11 9 6 5 4 ... 22 14 15 2 3 7 23
Attributes:
    units:    mm

In [19]:
meta.to_dataframe()

,region_area_m2,region_id
region,,
Eastern Hindu Kush,1.111471e+11,8
Western Himalaya,2.115142e+11,10
Eastern Himalaya,1.669436e+11,12
Central Himalaya,2.702681e+11,11
Karakoram,9.283049e+10,9
Western Pamir,1.248805e+11,6
Pamir Alay,8.344443e+10,5
Northern/Western Tien Shan,2.057249e+11,4
Dzhungarsky Alatau,3.954317e+10,1


In [20]:
# 6. Save to NetCDF
result_ds_mm.to_netcdf(r'D:/MSRSGI/Lab Rotation/Physical Geodesy/Python_Workspace/Organaized/Mascon/Combined/Result/Processed Data/New_HMA_regional_averages_WeightedMeanMascon_mm_End.nc')

In [21]:
# Load the dataset
ds = xr.open_dataset(r'D:/MSRSGI/Lab Rotation/Physical Geodesy/Python_Workspace/Organaized/Mascon/Combined/Result/Processed Data/New_HMA_regional_averages_WeightedMeanMascon_mm_End.nc')

# Rearrange dimensions to (time, region)
da = ds['lwe_thickness'].transpose('time', 'region')

# Convert to pandas DataFrame
df = da.to_pandas()  # time as index, region names as columns
df

region,Eastern Hindu Kush,Western Himalaya,Eastern Himalaya,Central Himalaya,Karakoram,Western Pamir,Pamir Alay,Northern/Western Tien Shan,Dzhungarsky Alatau,Western Kunlun Shan,...,Tibetan Interior Mountains,Tanggula Shan,Eastern Tibetan Mountains,Qilian Shan,Eastern Kunlun Shan,Altun Shan,Eastern Tien Shan,Central Tien Shan,Eastern Pamir,High Mountain Asia Region
time,,,,,,,,,,,,,,,,,,,,,
2002-04-30,84.776451,94.923691,14.021999,55.185547,21.293754,85.988134,75.257306,43.984829,55.467297,-36.876021,...,-31.337340,-29.337651,-40.520152,-8.136819,-50.010999,-7.495971,73.524231,55.858853,5.502060,11.673965
2002-05-31,105.582299,88.930113,35.833941,34.279438,27.315467,114.706547,95.334006,55.177431,63.232606,-32.897684,...,-38.665301,-31.332999,-45.161205,-16.137060,-52.789003,-12.054540,88.384062,81.236494,16.091122,13.475703
2002-06-30,53.622859,68.014173,95.820661,72.401195,5.863752,65.651325,56.945141,40.846422,50.551832,-33.160496,...,-28.686840,-22.121415,-37.027089,-9.679969,-44.365446,-10.330315,79.272083,76.970716,5.325396,19.348775
2002-07-31,-0.068562,46.401035,157.806937,111.793676,-16.303019,14.960929,17.276648,26.037713,37.448364,-33.432069,...,-18.375763,-12.602778,-28.621836,-3.007642,-35.661105,-8.548616,69.856371,72.562745,-5.799188,25.417615
2002-08-31,-53.759983,24.787897,219.793214,151.186158,-38.469791,-35.729467,-22.391845,11.229005,24.344897,-33.703642,...,-8.064686,-3.084141,-20.216583,3.664686,-26.956763,-6.766917,60.440659,68.154773,-16.923772,31.486456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-30,-99.268140,-82.187248,-206.499738,-179.914645,-37.070000,-132.999877,-105.577649,-79.273041,-53.914952,34.263106,...,2.879385,-69.859947,-3.595735,-12.287817,64.556469,34.731114,-106.424129,-139.051532,-27.928081,-62.168278
2015-12-31,-84.586468,-97.618076,-303.945217,-227.749875,-36.518008,-111.162782,-91.156589,-63.789832,-38.293400,30.558995,...,-10.605441,-87.948062,-17.863643,-15.839612,57.154069,30.107687,-95.642342,-130.412841,-28.181928,-79.390439
2016-01-31,-92.989767,-39.842604,-237.471756,-293.841697,-36.761059,-101.089923,-50.103936,-37.408384,-23.057803,20.887937,...,-31.509427,-149.426205,-30.697332,-17.796801,50.273274,34.108023,-122.380085,-163.580698,-15.689916,-80.076800


In [22]:
# Your region metadata
region_metadata_s = pd.DataFrame({
    'region': [
        'Eastern Hindu Kush', 'Western Himalaya', 'Eastern Himalaya', 'Central Himalaya', 'Karakoram',
        'Western Pamir', 'Pamir Alay', 'Northern/Western Tien Shan', 'Dzhungarsky Alatau', 'Western Kunlun Shan', 'Nyainqentanglha', 'Gangdise Mountains', 'Hengduan Shan',
        'Tibetan Interior Mountains', 'Tanggula Shan', 'Eastern Tibetan Mountains', 'Qilian Shan', 'Eastern Kunlun Shan', 'Altun Shan',
        'Eastern Tien Shan', 'Central Tien Shan', 'Eastern Pamir', 'High Mountain Asia Region'
    ],
    'region_id': [8, 10, 12, 11, 9, 6, 5, 4, 1, 13, 18, 17, 20, 16, 19, 21, 22, 14, 15, 2, 3, 7, 23]
})

# Sort by region_id
region_metadata_sorted_s = region_metadata_s.sort_values(by='region_id')
sorted_columns_s = region_metadata_sorted_s['region'].tolist()

df_s = df[sorted_columns_s]

df_s.index.name = 'time'
df_s.to_csv(r'D:/MSRSGI/Lab Rotation/Physical Geodesy/Python_Workspace/Organaized/Mascon/Combined/Result/Processed Data/New_EnsembleWeightedAverageMascon.csv', index=True)